# Purpose:
- Compare dff metrics between AAV injection pilots
- GCaMP8m RO vs icv
- GCaMP8s RO vs local
- GCaMP8m vs 8s
- GCaMP8m RO has 11 Hz, while all the rest has 44 Hz

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime as dt
from datetime import timezone as tz
import time
from glob import glob
import h5py
import imageio
import json
from pathlib import Path

from aind_codeocean_api.codeocean import CodeOceanClient
from aind_codeocean_api.credentials import CodeOceanCredentials

import capsule_utils
from comb.ophys_plane_grabber import OphysPlaneGrabber
from comb.ophys_plane_dataset import OphysPlaneDataset
from comb.behavior_ophys_dataset import BehaviorOphysDataset
from comb.behavior_session_grabber import BehaviorSessionGrabber
from comb.behavior_session_dataset import BehaviorSessionDataset

<ipython-input-1-5bbe4a9e1c8e>:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
scratch_dir = '/root/capsule/scratch/'
ribo_table = pd.read_csv(scratch_dir + 'ribo_table_with_depth_region.csv')
ribo_table.head()

,opid,depth,region,session_id,mouse_id,expression_strategy
0,1265760434,175,VISp,1265619832,667825,RiboL1-GCaMP8m(icv)
1,1265760436,275,VISp,1265619832,667825,RiboL1-GCaMP8m(icv)
2,1263579241,175,VISp,1262862975,667825,RiboL1-GCaMP8m(icv)
3,1263579243,275,VISp,1262862975,667825,RiboL1-GCaMP8m(icv)
4,1267294087,275,VISp,1267183073,667826,RiboL1-GCaMP8m(icv)


In [3]:
data_dir = Path('/root/capsule/data')
# get subfolders with opid 
opid = 1265760434
subfolders = [f for f in data_dir.iterdir() if f.is_dir() and f'{opid}' in f.name]
subfolders

[]

In [7]:
glob(str(data_dir / f'*processed*/{opid}'))

['/root/capsule/data/multiplane-ophys_667825_2023-04-21_13-23-24_processed_2024-02-20_18-23-08/1265760434']

In [9]:
glob(str(data_dir / f'**/pophys/**{opid}'))

['/root/capsule/data/multiplane-ophys_667825_2023-04-21_13-23-24/pophys/ophys_experiment_1265760434']

In [10]:
def fetch_opd(opid):
    data_dir = Path('/root/capsule/data')
    opid_path = glob(str(data_dir / f'**/{opid}'))
    raw_path = glob(str(data_dir / f'**/pophys/**{opid}'))
    if (len(opid_path) == 0) or (len(raw_path) == 0):
        print(f'No data found for opid {opid}')
        return None
    else:
        print(f'Load {opid}')
        plane_path = opid_path[0]
        raw_path = raw_path[0]
        return OphysPlaneDataset(plane_path, raw_path)
        
    

In [12]:
opd = fetch_opd(1265760434)

Load 1265760434
Currently sync file stored in raw data assest, will load since raw_folder_path is provided (02/01/2024)


In [17]:

mc_movie_fn = opd.file_paths['motion_transform_csv'].parent / f'{opd.opid}_registered.h5'
framerate = capsule_utils.get_rig_acquisition_frame_rate(opd)
num_frames = int(framerate * 60 * 5)
with h5py.File(mc_movie_fn, 'r') as f:
    mc_movie = f['data'][-num_frames:]
save_dir = '/root/capsule/results/'    
save_fn = save_dir + f'{opd.opid}_mc_movie_last_5min.tiff'
imageio.mimwrite(save_fn, mc_movie)

In [ ]:
save_dir = '/root/capsule/results/'    
save_fn = save_dir + f'{opd.opid}_mc_movie_last_5min.tiff'
imageio.mimwrite(save_fn, mc_movie)

In [13]:
opd.file_paths

{'platform_json': PosixPath('/root/capsule/data/multiplane-ophys_667825_2023-04-21_13-23-24_processed_2024-02-20_18-23-08/1265760434/1265619832_platform.json'),
 'processing_json': PosixPath('/root/capsule/data/multiplane-ophys_667825_2023-04-21_13-23-24_processed_2024-02-20_18-23-08/1265760434/processing.json'),
 'params_json': PosixPath('/root/capsule/data/multiplane-ophys_667825_2023-04-21_13-23-24_processed_2024-02-20_18-23-08/1265760434/events/1265760434_params.json'),
 'registered_metrics_json': PosixPath('/root/capsule/data/multiplane-ophys_667825_2023-04-21_13-23-24_processed_2024-02-20_18-23-08/1265760434/movie_qc/1265760434_registered_metrics.json'),
 'output_json': PosixPath('/root/capsule/data/multiplane-ophys_667825_2023-04-21_13-23-24_processed_2024-02-20_18-23-08/1265760434/segmentation/segmentation_output.json'),
 'average_projection_png': PosixPath('/root/capsule/data/multiplane-ophys_667825_2023-04-21_13-23-24_processed_2024-02-20_18-23-08/1265760434/motion_correction